In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

import numpy as np

#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn = psycopg2.connect("host=127.0.0.1 dbname=udacity_p1 user=dar1en password=")
cur = conn.cursor()

def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [2]:
filepath = "data/song_data"
song_files = get_files(filepath)

In [2]:
filepath = "data/song_data"
song_files = get_files(filepath)

for each_song_file in song_files:
    df = pd.read_json(each_song_file,typ='series')
    song_data = df[['song_id','title','artist_id','year','duration']]
    cur.execute(song_table_insert, song_data)
    conn.commit()
    artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
    artist_data.reindex()
    cur.execute(artist_table_insert, artist_data)
    conn.commit()

In [3]:
log_filepath = "data/log_data"
log_files = get_files(log_filepath)
for each_log_files in log_files:
    df_log=pd.read_json(each_log_files,lines=True)
    df = df_log.loc[df_log['page'] == 'NextSong']
    t = pd.to_datetime(df['registration'], unit='ms')
    x=t.to_frame()
    time_data=np.transpose(np.array \
                       ([x["registration"].values,\
                         t.dt.hour.values,\
                         t.dt.day.values, \
                         t.dt.week.values,\
                         t.dt.month.values, \
                         t.dt.year.values, \
                         t.dt.weekday.values]))
    column_labels = ('registration','hour','day','week','month','year','weekday')
    time_df =  pd.DataFrame(data=time_data, columns=column_labels)
    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()
    user_df = df_log[['userId','firstName','lastName','gender','level']]
    for i, row in user_df.iterrows():
        if(row[0]!=''):
            cur.execute(user_table_insert, row)
            conn.commit()

    for index, row in df.iterrows():
        cur.execute(song_select,(row.song,row.artist,row.length))
        results=cur.fetchone()
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
        songplay_data = (index,row.registration,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

In [4]:
conn.close()